<a href="https://colab.research.google.com/github/anjaliasha123/AirlineTweetClassification/blob/main/src/BiLSTMCNNAtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Initial required libraries

In [16]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
#intializing global variables
data = pd.read_csv('https://raw.githubusercontent.com/anjaliasha123/Data_Visualization_projects/master/Streamlit_python/Tweets.csv')
Xdata = data[['airline_sentiment_confidence','airline','text','latitude','longitude']]
Ydata = data['airline_sentiment']

In [18]:
#suffling the data and forming a balanced data set
num_tweets_per_category = 2363
shuffled = data.reindex(np.random.permutation(data.index))
positive = shuffled[shuffled['airline_sentiment'] == 'positive'][:num_tweets_per_category]
negative = shuffled[shuffled['airline_sentiment'] == 'negative'][:num_tweets_per_category]
neutral  = shuffled[shuffled['airline_sentiment'] == 'neutral'][:num_tweets_per_category]
data  = pd.concat([positive,negative,neutral], ignore_index=True)

In [19]:
Xdata = data[['airline_sentiment_confidence','airline','text','latitude','longitude']]
Ydata = data['airline_sentiment']

In [20]:
Xdata.head(10)

,airline_sentiment_confidence,airline,text,latitude,longitude
0,0.6986,Southwest,@SouthwestAir @love_dragonss oh my god LAUREN ...,46.709294,-97.337366
1,1.0000,Virgin America,"@VirginAmerica thank goodness!! Also, see you ...",41.411314,-119.153471
2,1.0000,United,@united Resolved. Over hour of work on ground...,37.626267,-119.172956
3,1.0000,US Airways,@USAirways ok thank you! Very helpful! This is...,40.758600,-73.973100
4,0.6669,Delta,@JetBlue thanks...,35.846756,-98.168765
5,1.0000,Delta,@JetBlue thanks so much for your condolences a...,39.383085,-99.074612
6,0.7139,Delta,@JetBlue #flyfi thank you! Seattle and #UDUB h...,38.308253,-98.744859
7,1.0000,Southwest,@SouthwestAir Left my computer on the plane. T...,43.487804,-94.652825
8,1.0000,Southwest,@SouthwestAir THANK YOU for finally making you...,43.487754,-100.334976
9,0.6867,Southwest,@SouthwestAir @Imaginedragons @beatsmusic well...,39.736510,-101.519388


In [21]:
Ydata = Ydata.to_frame()
Ydata.head(2)

,airline_sentiment
0,positive
1,positive


In [22]:
most_common = 20000 #number of vocabs to be considered for the embedding matrix
max_len = 100 
epochs = 10
embed_dim = 100
batch_size = 100

#Function to create word embeddings

- we use pre-trained glove model to create word2vec 
- create a embedding matrix

In [23]:
from utilsfunction  import dataPreprocessing

In [24]:
em, X, y, num = dataPreprocessing(Xdata,Ydata, most_common=most_common, max_len= max_len, embed_dim=embed_dim)

Found 10412 unique tokens.


 Labels:  ['positive', 'neutral', 'negative']
airline_sentiment
positive             2363
neutral              2363
negative             2363
dtype: int64
  airline_sentiment  label
0          positive      1
1          positive      1
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Text 1:  @SouthwestAir @love_dragonss oh my god LAUREN OH MY GOD OH MY GOD
Text 1 converted to word index sequences:  [11, 85, 1154, 340, 13, 1087, 1375, 340, 13, 1087, 340, 13, 1087]
Text 1 word to padded index sequence of length 100: 
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   11   85 1154  34

#Creating the Model

In [25]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D,GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import  SGD, Adam

In [28]:
#creating the embedding layer
embedding_layer = Embedding(input_dim=num,output_dim=max_len,weights=[em],input_length=max_len,trainable=False)
#input_layer
input_layer = Input((max_len,),name='InputLayer')
#embedding layer
embedding_layer = embedding_layer(input_layer)
#bi-directional LSTM layer
lstm = Bidirectional(LSTM(50,return_sequences=True))(embedding_layer)
#dropout layer
drop_lstm = Dropout(0.3)(lstm)
#CNN layers
first_conv_layer = Conv1D(128, 3, activation='relu')(lstm)
first_max_pooling_layer = MaxPooling1D(3)(first_conv_layer)
second_conv_layer = Conv1D(128, 4, activation='relu')(first_conv_layer)
second_max_pooling_layer = MaxPooling1D(4)(second_conv_layer)
third_conv_layer = Conv1D(128, 5, activation='relu')(second_max_pooling_layer)
third_max_pooling_layer = MaxPooling1D(5)(third_conv_layer)
merged = Concatenate(axis=1)([first_max_pooling_layer,second_max_pooling_layer,third_max_pooling_layer])
context_vector = GlobalMaxPool1D()(merged)

#attention mechanism
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(100, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)
hidden_size = int(lstm.shape[2])
h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(lstm)
pre_activation = Concatenate(axis=1)([context_vector, h_t])
e = densor1(pre_activation)
energies = densor2(e)
alphas = activator(energies)
s = dotor([alphas, lstm])

#hidden_size = int(lstm.shape[2])
#h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(lstm)
#pre_activation = Concatenate(axis=1)([merged_vector, h_t])
output_layer = Dense(3, use_bias=False, activation='softmax', name='attention_vector')(s)
model = Model(input_layer,output_layer)
model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.01),
        metrics=['accuracy']
             )

In [29]:
#splitting the data as 80 : 20
# 80% for training and 20% for testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
import tensorflow

In [32]:
from tensorflow.keras.callbacks import Callback

class MyThresholdCallback(tensorflow.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        acc = logs["accuracy"]
        if acc >= self.threshold:
            self.model.stop_training = True


callback=MyThresholdCallback(threshold=0.88)
r = model.fit(
            X_train,
            y_train,
            batch_size=50,
            epochs=200,
            validation_split=0.2,
            callbacks=[callback]  
            )



Epoch 1/200
91/91 [==============================] - 20s 219ms/step - loss: 0.6496 - accuracy: 0.7262 - val_loss: 0.7019 - val_accuracy: 0.7110
Epoch 2/200
91/91 [==============================] - 20s 216ms/step - loss: 0.5628 - accuracy: 0.7705 - val_loss: 0.6888 - val_accuracy: 0.7093
Epoch 3/200
91/91 [==============================] - 20s 218ms/step - loss: 0.4744 - accuracy: 0.8186 - val_loss: 0.6889 - val_accuracy: 0.7101
Epoch 4/200
91/91 [==============================] - 20s 217ms/step - loss: 0.3934 - accuracy: 0.8459 - val_loss: 0.7701 - val_accuracy: 0.7119
Epoch 5/200
91/91 [==============================] - 20s 216ms/step - loss: 0.3053 - accuracy: 0.8862 - val_loss: 0.8335 - val_accuracy: 0.7137


In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
InputLayer (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 100, 100)     1041300     InputLayer[0][0]                 
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 100)     60400       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 98, 128)      38528       bidirectional[0][0]              
______________________________________________________________________________________________

In [34]:
model.save("bilstmCNNAtt.h5")